# 02 — PyTorch Implementation (No Autograd)

## 1. Imports & Deterministic Seeds
## 2. Notebook Outline
## 3. Synthetic Dataset (reuse from Notebook 01)
## 4. PyTorch Tensors for Model Parameters (requires_grad = False)
## 5. Forward Pass (PyTorch)
## 6. Loss Function (PyTorch)
## 7. Numerical Consistency Check (NumPy vs PyTorch)
## 8. Conclusion — Ready for Autograd Notebook
